In [15]:
import torch
import gym as g
from gym import spaces
from connect4 import *
from envs import ConnectNEnv
from networks.architecture import RepresentationNetwork, DynamicsNetwork, PredictionNetwork
import numpy as np

In [16]:
from architecture.engine import *
from architecture.network import MuZeroNetwork

In [17]:
muzeronet = MuZeroNetwork(3, 42, 7)

In [18]:
# Test representation
image = torch.rand((1,3,6,7))
print(image)
encoded_state = muzeronet.representation(image)

tensor([[[[0.8164, 0.3978, 0.5306, 0.7621, 0.6582, 0.9760, 0.9334],
          [0.7042, 0.7333, 0.5411, 0.4834, 0.8974, 0.8145, 0.9633],
          [0.0543, 0.8457, 0.8538, 0.9461, 0.5980, 0.2447, 0.8905],
          [0.1447, 0.6893, 0.7165, 0.2817, 0.7685, 0.0572, 0.3851],
          [0.9205, 0.3805, 0.7149, 0.2021, 0.1950, 0.6012, 0.4564],
          [0.5681, 0.0198, 0.1473, 0.3247, 0.9509, 0.5057, 0.1253]],

         [[0.1627, 0.4590, 0.7942, 0.7758, 0.4085, 0.8357, 0.0628],
          [0.1283, 0.7485, 0.3682, 0.2006, 0.5233, 0.4437, 0.8897],
          [0.7680, 0.4537, 0.4231, 0.5720, 0.8746, 0.1214, 0.1399],
          [0.2545, 0.7950, 0.8608, 0.9635, 0.7578, 0.8966, 0.1168],
          [0.6715, 0.5328, 0.6477, 0.7215, 0.1433, 0.7837, 0.8096],
          [0.1592, 0.0315, 0.1015, 0.1938, 0.3604, 0.0491, 0.4752]],

         [[0.8990, 0.1772, 0.1149, 0.0940, 0.5202, 0.1769, 0.0163],
          [0.2009, 0.8592, 0.5997, 0.4336, 0.9139, 0.9262, 0.8298],
          [0.6779, 0.1890, 0.9911, 0.3530, 0

In [19]:
# Test dynamics
action = torch.Tensor([2]).view(1,1)
print(action, action.shape)
muzeronet.dynamics(encoded_state=encoded_state, action=action)

tensor([[2.]]) torch.Size([1, 1])


(tensor([[[[4.3214e-01, 4.0599e-02, 2.8226e-01,  ..., 1.1668e-01,
            0.0000e+00, 1.3287e-01],
           [4.7820e-01, 5.1756e-01, 4.1466e-01,  ..., 3.9560e-01,
            7.6381e-02, 3.6691e-01],
           [4.4407e-02, 1.5847e-01, 0.0000e+00,  ..., 7.5680e-01,
            5.7044e-01, 8.8602e-01],
           [3.5920e-01, 2.5235e-01, 1.0000e+00,  ..., 7.9974e-01,
            8.2152e-02, 5.0499e-02],
           [3.7395e-01, 1.9091e-01, 0.0000e+00,  ..., 7.5617e-01,
            0.0000e+00, 1.5508e-01],
           [3.9213e-01, 3.3644e-01, 0.0000e+00,  ..., 5.4355e-01,
            2.6583e-01, 6.9747e-01]],
 
          [[6.1616e-02, 6.5698e-01, 7.0193e-02,  ..., 5.4003e-01,
            6.0881e-01, 3.5305e-01],
           [5.8537e-02, 6.2559e-01, 9.0076e-02,  ..., 2.5636e-01,
            2.7658e-01, 9.9618e-02],
           [0.0000e+00, 0.0000e+00, 3.3487e-01,  ..., 2.6932e-01,
            4.2478e-01, 2.2559e-01],
           [5.4717e-04, 1.4172e-01, 4.7592e-01,  ..., 5.8897e-01,
    

In [20]:
# Test prediction
muzeronet.prediction(encoded_state=encoded_state)

(tensor([[ 3.8739e-01, -4.1028e-01, -4.9489e-01,  4.8979e-01, -2.4874e-01,
          -2.8344e-01,  3.9383e-01,  2.0841e-02,  1.2388e-01, -1.4315e-01,
          -3.6518e-01, -4.2270e-01,  2.3656e-02,  2.7098e-01, -1.5509e-01,
           2.3380e-01, -8.8022e-01, -1.8085e-01, -3.7263e-01,  5.7183e-01,
          -3.0691e-01, -4.4679e-01,  1.0563e-01, -3.3149e-01,  6.1416e-01,
          -1.5688e-01,  3.1850e-01, -6.4031e-01,  8.1237e-01,  4.7470e-01,
           1.1176e+00, -2.6204e-01,  4.1129e-01,  1.5288e-01,  6.6225e-01,
          -3.9296e-01, -1.1490e-02, -2.3583e-02,  2.2654e-01, -4.8640e-02,
           8.4233e-02,  7.5906e-01, -2.3497e-01,  4.1439e-01,  4.0514e-02,
           1.0399e-01, -3.1816e-02,  6.6869e-02,  2.8041e-01,  1.2584e-01,
           3.2183e-01, -5.6122e-02, -4.8337e-01, -2.8260e-01, -2.2473e-01,
          -2.4019e-01, -2.9997e-01, -1.9957e-01,  6.8398e-02,  2.1050e-01,
          -5.3910e-01,  4.6702e-01,  3.7780e-01,  6.5241e-01,  4.8683e-01,
           7.4357e-01, -8

In [21]:
# Test network units
muzeronet.recurrent_inference(encoded_state, action)
muzeronet.initial_inference(image).policy_logits

tensor([[ 0.0334, -0.2389, -0.2324,  0.0448, -0.9867,  0.1865,  0.1319]],
       grad_fn=<AddmmBackward0>)

In [22]:
muzeronet.from_support_to_scalar(muzeronet.initial_inference(image).reward, muzeronet.prediction_support_size)

tensor([[-0.]])

In [23]:
# Testing MinMax

minmax = MinMaxStats((1,3)) # minmax = MinMaxStats()
minmax.normalize(10)
# minmax.update(1)
# minmax.update(3)
minmax.normalize(190)

94.5

In [24]:
# testing Muzero config
from envs import ConnectNEnv

muzeroconfig = MuZeroConfig(ConnectNEnv, {})
muzeroconfig.new_game()
muzeroconfig.temperature_value(30)

0.0

In [25]:
# Testing atomic functions
node = Node(0)
node1 = Node(0)
node1.visit_count = 10
node2 = Node(0.1)
node2.visit_count = 20
node.children = {1: node1, 2: node2}
select_action(muzeroconfig, 20, node, muzeronet)

2

In [13]:
from architecture.network import MuZeroConnectN

muzeronet = MuZeroConnectN()
play_game(muzeroconfig, muzeronet)

KeyboardInterrupt: 

In [27]:
from architecture.replay import ReplayBuffer

In [ ]:
from typing import List, Tuple
from torch import Tensor

class MuZero:

    def __init__(self, config: MuZeroConfig):
        self.steps = 0
        self.config = config
        self.network = self.load_network()

    def execute(self):
        replay_buffer = ReplayBuffer(self.config.max_buffer_size, self.config.batch_size)

        while True:
            self.self_play()
            self.train_model()

    def self_play(self):
        pass

    def train_model(self, replay_buffer: ReplayBuffer):
        network = self.get_latest_network()
        optimizer = torch.optim.SGD(network.parameters(), lr=self.config.lr, momentum=self.config.momentum)
        scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer=optimizer, lr_lambda=self.config.decaying_func())

        for i in self.config.training_steps:
            if i%self.config.step_for_saving == 0:
                self.save_network()
            batch = replay_buffer.sample_batch(self.config.unroll_steps, self.config.td_steps)
            self.update_weights(batch, optimizer, network, scheduler)

    def update_weights(self, batch: List[Tuple[Tensor, List, List]], optimizer: torch.optim.Optimizer, network: MuZeroNetwork, scheduler: torch.optim.lr_scheduler.LRScheduler = None):

        loss = 0
        for image, actions, targets in batch:
            value, reward, policy_logits, hidden_state = network.initial_inference(image).unpack()
            # Grab all predictions as a tuple of: 
            # - gradient_scale (based on the number of actions)
            # - predicted value
            # - predicted logits
            # - predicted reward
            predictions: List[Tuple[float, Tensor, Tensor, Tensor]] = [(1.0, value, policy_logits, reward)]

            for action in actions:
                value, reward, policy_logits, hidden_state = network.recurrent_inference(hidden_state=hidden_state, action=action).unpack()

                # Scale the gradient
                predictions.append((1.0/len(actions), value, policy_logits, reward))
                # Adjust hidden_state gradient
                hidden_state = ut.scale_torch_gradient(hidden_state, self.config.gradient_scale_factor)

            for pred, target in zip(predictions, targets):

                l = self.calculate_loss(pred, target)

    def calculate_loss(self, pred: List[Tuple[float, Tensor, Tensor, Tensor]], target):
        scaling_factor, value, policy_logits, reward = pred
        target_value, target_reward, target_policy = target


    def get_latest_network(self):
        network = MuZeroConnectN()
        self.load_network(network)
        return network
        
    def load_network(self, network: MuZeroNetwork):
        loaded_nets = network.load_network()

    def save_network(self, network: MuZeroNetwork):
        network.save_network()





    def train_network_on_batch(self, network_optim: torch.optim.Optimizer, muzero_network: MuZeroNetwork):
